In [8]:
import os
from py4j.java_gateway import JavaGateway
from scipy.optimize import minimize


gateway = JavaGateway()   # connect to the JVM

scriptdir = os.getcwd()


model_params = None   # Model paremeter vector. Only used for fast cloning
cost_funtion = None   # OpenDA cost function

In [9]:
import calibrate_swan
def py_list_to_j_array(py_x):
    """
    Create a java double array from python list

    :param py_x: python list of doubles
    :return: java array of doubles copy of py_x
    """
    n = len (py_x)
    double_class = gateway.jvm.double
    j_x = gateway.new_array(double_class, n)

    for i in range(n):
        j_x[i]=py_x[i]

    return j_x

def j_array_to_py_list(j_x):
    """
    Create a python list from a java array
    :param j_x: java array
    :return: python list with values of j_x
    """

    n = len(j_x)
    py_x =[None]*n
    for i in range(n):
        py_x[i]=j_x[i]

    return py_x


def initialize_openda_configurable(openda_configurable, input_dir, config_xml):
    """
    Initialise an OpenDA configurable (e.g. ModelFactory, StochObserver)

    :param openda_configurable: OpenDA object implementing IConfigurable
    :param input_dir: Working directory/input directory
    :param config_xml: Configuration file
    :return:
    """

    # Translate the input strings to java objects (File and String array)
    j_input_dir = gateway.jvm.java.io.File(input_dir)
    string_class = gateway.jvm.String
    j_arguments = gateway.new_array(string_class, 1)
    j_arguments[0] = config_xml

    # Initialize ..
    openda_configurable.initialize(j_input_dir, j_arguments)



In [10]:
model_input_dir = os.path.join(scriptdir, 'swanModel', 'config')
model_config_xml = "swanStochModelConfig.xml"


model_factory = gateway.jvm.org.openda.model_swan.SwanCalibStochModelFactory()
initialize_openda_configurable(model_factory, model_input_dir, model_config_xml)

#Initialize stoch observer
#observer_input_dir = "/Users/nils/Develop/py4j/stochObserver"
observer_input_dir = model_input_dir = os.path.join(scriptdir, 'stochObserver')
observer_config_xml = "swanStochObsConfig.xml"

observer = gateway.jvm.org.openda.observers.IoObjectStochObserver()
initialize_openda_configurable(observer, observer_input_dir, observer_config_xml)

#Initialize cost function org.openda.algorithms
cost_funtion = gateway.jvm.org.openda.algorithms.SimulationKwadraticCostFunction(model_factory, observer)


#Get initial parameter
outputLevel = gateway.jvm.org.openda.interfaces.IStochModelFactory.OutputLevel.Debug
model_ini = model_factory.getInstance(outputLevel)
p = model_ini.getParameters()
model_params= p

In [12]:
x=model_ini.getParameters();


In [17]:
print (j_array_to_py_list(x.getValues()))


[0.05, 2.5]


In [18]:
x2=model_ini.getState();

Py4JJavaError: An error occurred while calling o8.getState.
: java.lang.UnsupportedOperationException: org.openda.model_swan.SwanCalibWrapper: Method not implemented yet.
	at org.openda.model_swan.SwanCalibWrapper.getState(SwanCalibWrapper.java:427)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:214)
	at java.lang.Thread.run(Thread.java:748)
